<a href="https://colab.research.google.com/github/VishhAAkha/SNN/blob/main/SNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras import layers, models

In [3]:
mnist = tf.keras.datasets.mnist

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Display the shapes of the datasets

In [6]:
print("Training data shape:", x_train.shape, y_train.shape)
print("Test data shape:", x_test.shape, y_test.shape)

Training data shape: (60000, 28, 28) (60000,)
Test data shape: (10000, 28, 28) (10000,)


# Normalize the input data

In [7]:
x_train, x_test = x_train / 255.0, x_test / 255.0

# Build the model


In [8]:
model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])


# Define a simple neural network model

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])


# Compile the model

In [10]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model

In [11]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 1.2530 - accuracy: 0.6898
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.5650 - accuracy: 0.8435
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4384 - accuracy: 0.8757
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3855 - accuracy: 0.8890
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3530 - accuracy: 0.8982


# Evaluate the model

In [12]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

313/313 [==============================] - 1s 2ms/step - loss: 0.3005 - accuracy: 0.9143
Loss: 0.30052393674850464
Accuracy: 0.9143000245094299


2. Deploy the Model to GPU

In [13]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


#Step 3: Perform Quantization

Install Hugging Face Transformers:

In [14]:
!pip install transformers


In [15]:
!pip install --upgrade transformers


In [19]:
!pip install --upgrade transformers
!pip install peft


In [22]:
!pip install --upgrade transformers
!pip install peft
!pip install torch


In [23]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import LoraConfig, get_peft_model
import torch


Load the Pretrained Model and Tokenizer:

In [24]:
model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenize Input Text:

In [25]:
text = "Hello, how are you?"
inputs = tokenizer(text, return_tensors='pt')


Perform Inference with the Original Model:

In [26]:
outputs = model(**inputs)
print("Original model output:", outputs.logits)


Original model output: tensor([[-0.0234,  0.1575]], grad_fn=<AddmmBackward0>)


In [30]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import LoraConfig, get_peft_model
import torch


Load the Pretrained Model and Tokenizer:

In [31]:
model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenize Input Text:

In [32]:
text = "Hello, how are you?"
inputs = tokenizer(text, return_tensors='pt')


Perform Inference with the Original Model:

In [33]:
outputs = model(**inputs)
print("Original model output:", outputs.logits)


Original model output: tensor([[-0.0491,  0.0185]], grad_fn=<AddmmBackward0>)


Apply LoRA to the Model with Correct target_modules:

In [34]:
lora_config = LoraConfig(
    r=4,
    lora_alpha=32,
    target_modules=["attention.q_lin", "attention.v_lin"],  # Adjust target modules for distilbert
    lora_dropout=0.1,
    bias="lora_only"
)

# Add LoRA layers to the model
model = get_peft_model(model, lora_config)

# Perform inference with the LoRA-applied model
outputs = model(**inputs)
print("LoRA-applied model output:", outputs.logits)


LoRA-applied model output: tensor([[-0.0491,  0.0185]], grad_fn=<AddmmBackward0>)


For QLoRA Quantization:

In [35]:
from peft import QLoraConfig

# Apply QLoRA to the model
qlora_config = QLoraConfig(
    r=4,
    lora_alpha=32,
    target_modules=["attention.q_lin", "attention.v_lin"],  # Adjust target modules for distilbert
    lora_dropout=0.1,
    bias="lora_only"
)

# Add QLoRA layers to the model
model = get_peft_model(model, qlora_config)

# Perform inference with the QLoRA-applied model
outputs = model(**inputs)
print("QLoRA-applied model output:", outputs.logits)


ImportError: cannot import name 'QLoraConfig' from 'peft' (/usr/local/lib/python3.10/dist-packages/peft/__init__.py)

Apply LoRA to the Model:

In [40]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

# Define LoRA configuration and custom layers
class LoRALayer(tf.keras.layers.Layer):
    def __init__(self, units, rank):
        super(LoRALayer, self).__init__()
        self.units = units
        self.rank = rank

    def build(self, input_shape):
        self.kernel = self.add_weight("kernel", shape=[int(input_shape[-1]), self.units])
        self.lora_a = self.add_weight("lora_a", shape=[int(input_shape[-1]), self.rank])
        self.lora_b = self.add_weight("lora_b", shape=[self.rank, self.units])

    def call(self, inputs):
        lora_output = tf.matmul(inputs, self.lora_a)
        lora_output = tf.matmul(lora_output, self.lora_b)
        return tf.matmul(inputs, self.kernel) + lora_output

# Create a new model with LoRA layers
def create_lora_model():
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        LoRALayer(128, rank=4),
        Dense(10, activation='softmax')
    ])
    return model

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Create, compile and train the LoRA model
lora_model = create_lora_model()
lora_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
lora_model.fit(x_train, y_train, epochs=5)
lora_model.evaluate(x_test, y_test)



Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3624 - accuracy: 0.8981
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3043 - accuracy: 0.9154
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2918 - accuracy: 0.9182
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2856 - accuracy: 0.9207
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.2982 - accuracy: 0.9154


[0.29822367429733276, 0.9154000282287598]

QLoRA Quantization Example:

In [38]:
class QLoRALayer(tf.keras.layers.Layer):
    def __init__(self, units, rank, bits=8):
        super(QLoRALayer, self).__init__()
        self.units = units
        self.rank = rank
        self.bits = bits

    def build(self, input_shape):
        self.kernel = self.add_weight("kernel", shape=[int(input_shape[-1]), self.units])
        self.lora_a = self.add_weight("lora_a", shape=[int(input_shape[-1]), self.rank])
        self.lora_b = self.add_weight("lora_b", shape=[self.rank, self.units])

    def call(self, inputs):
        lora_output = tf.matmul(inputs, self.lora_a)
        lora_output = tf.matmul(lora_output, self.lora_b)
        quantized_kernel = tf.quantization.fake_quant_with_min_max_vars(self.kernel, min=-1.0, max=1.0, num_bits=self.bits)
        return tf.matmul(inputs, quantized_kernel) + lora_output

# Create a new model with QLoRA layers
def create_qlora_model():
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        QLoRALayer(128, rank=4, bits=8),
        Dense(10, activation='softmax')
    ])
    return model

qlora_model = create_qlora_model()

# Compile and train the QLoRA model
qlora_model.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

qlora_model.fit(x_train, y_train, epochs=5)
qlora_model.evaluate(x_test, y_test)


Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.3641 - accuracy: 0.8953
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3030 - accuracy: 0.9157
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2916 - accuracy: 0.9194
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2847 - accuracy: 0.9211
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.2911 - accuracy: 0.9198


[0.2911244332790375, 0.9197999835014343]

Step 5: Compare Results

In [39]:
# Define the table structure
import pandas as pd

data = {
    "Model": ["Original ANN", "LoRA ANN", "QLoRA ANN"],
    "Accuracy": [
        model.evaluate(x_test, y_test, verbose=0)[1],
        lora_model.evaluate(x_test, y_test, verbose=0)[1],
        qlora_model.evaluate(x_test, y_test, verbose=0)[1]
    ]
}

# Create DataFrame and display
df = pd.DataFrame(data)
print(df)



AttributeError: 'DistilBertForSequenceClassification' object has no attribute 'evaluate'